# Athena.ipynb

## Problem 1: Get top 10 subreddits by count of comments

The response should contain two columns: `subreddit` and `comment_count`.
Save the results in a file called `prob1_results.csv`. This needs to be commited to the repo.

In [1]:
# libraries  
import os
import time
import boto3
import pandas as pd
from typing import Dict
import logging

In [2]:
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
SCHEMA_NAME = "schema_name"

# fill in your net id below.
netid = "ek976"
S3_STAGING_PREFIX = "data/results"
S3_BUCKET_NAME = f"athena-{netid}"
S3_STAGING_DIR = f"s3://{S3_BUCKET_NAME}/{S3_STAGING_PREFIX}/"
S3_OUTPUT_DIRECTORY = "data"
AWS_REGION = "us-east-1"

In [4]:
def download_and_load_query_results(
    client: boto3.client, query_response: Dict
) -> pd.DataFrame:
    logger.info("download_and_load_query_results, enter")
    while True:
        try:
            # This function only loads the first 1000 rows
            client.get_query_results(
                QueryExecutionId=query_response["QueryExecutionId"]
            )
            break
        except Exception as err:
            if "not yet finished" in str(err):
                time.sleep(0.001)
            else:
                raise err
    logger.info(f"Time to complete query: {time.time() - start_time}s")
    temp_file_location: str = "athena_query_results.csv"
    s3_client = boto3.client(
        "s3",
        region_name=AWS_REGION,
    )
    s3_path = os.path.join(S3_STAGING_PREFIX, f"{query_response['QueryExecutionId']}.csv")
    logger.info(f"downloading file from S3_BUCKET_NAME={S3_BUCKET_NAME}, s3_path={s3_path}, to local file {temp_file_location}")
    s3_client.download_file(
        S3_BUCKET_NAME,
        s3_path,
        temp_file_location,
    )
    df = pd.read_csv(temp_file_location)
    logger.info(f"results dataframe shape is {df.shape}")
    return df

In [5]:
athena_client = boto3.client(
    "athena",
    region_name=AWS_REGION,
)

In [6]:
%%time
start_time = time.time()
q = 'SELECT count(*) from "AwsDataCatalog"."a05"."a05";'
response = athena_client.start_query_execution(
    QueryString=q,
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)
logger.info(response)
df_data = download_and_load_query_results(athena_client, response)

logger.info(df_data.head())
logger.info(f"Data fetched in {time.time() - start_time}s")

[2024-11-11 03:00:53,473] p537 {<timed exec>:11} INFO - {'QueryExecutionId': 'd691202c-c9be-43a7-bc60-e6108b85a745', 'ResponseMetadata': {'RequestId': 'aa188a28-d128-43d2-b651-604ef6233729', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Mon, 11 Nov 2024 03:00:53 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '59', 'connection': 'keep-alive', 'x-amzn-requestid': 'aa188a28-d128-43d2-b651-604ef6233729'}, 'RetryAttempts': 0}}
[2024-11-11 03:00:53,475] p537 {1241576256.py:4} INFO - download_and_load_query_results, enter
[2024-11-11 03:00:54,479] p537 {1241576256.py:17} INFO - Time to complete query: 1.1928610801696777s
[2024-11-11 03:00:54,647] p537 {1241576256.py:24} INFO - downloading file from S3_BUCKET_NAME=athena-ek976, s3_path=data/results/d691202c-c9be-43a7-bc60-e6108b85a745.csv, to local file athena_query_results.csv
[2024-11-11 03:00:54,743] p537 {1241576256.py:31} INFO - results dataframe shape is (1, 1)
[2024-11-11 03:00:54,746] p537 {<timed exec>:14} INF

CPU times: user 235 ms, sys: 37.1 ms, total: 273 ms
Wall time: 1.47 s


In [7]:
# Query to get all column names from the specified table
q = '''
SELECT column_name 
FROM information_schema.columns 
WHERE table_schema = 'a05' 
AND table_name = 'a05';
'''

response = athena_client.start_query_execution(
    QueryString=q,
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

logger.info(response)
df_columns = download_and_load_query_results(athena_client, response)

output_file = "df_columns.csv"
df_columns.to_csv(output_file, index=False)
logger.info(f"{output_file}")

logger.info(f"Data fetched in {time.time() - start_time}s")


[2024-11-11 03:00:58,233] p537 {1370647446.py:18} INFO - {'QueryExecutionId': 'f9d29b0b-6f78-414d-b70f-b9eeca7f5b87', 'ResponseMetadata': {'RequestId': 'dc265bd0-052a-4c47-bb32-fada842359a8', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Mon, 11 Nov 2024 03:00:58 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '59', 'connection': 'keep-alive', 'x-amzn-requestid': 'dc265bd0-052a-4c47-bb32-fada842359a8'}, 'RetryAttempts': 0}}
[2024-11-11 03:00:58,240] p537 {1241576256.py:4} INFO - download_and_load_query_results, enter
[2024-11-11 03:00:58,870] p537 {1241576256.py:17} INFO - Time to complete query: 5.5844526290893555s
[2024-11-11 03:00:58,883] p537 {1241576256.py:24} INFO - downloading file from S3_BUCKET_NAME=athena-ek976, s3_path=data/results/f9d29b0b-6f78-414d-b70f-b9eeca7f5b87.csv, to local file athena_query_results.csv
[2024-11-11 03:00:58,989] p537 {1241576256.py:31} INFO - results dataframe shape is (17, 1)
[2024-11-11 03:00:59,011] p537 {1370647446.py:23} 

## Problem 1 

In [8]:
# Problem 1: Get top 10 subreddits by count of comments
q = '''
SELECT subreddit, COUNT(id) as comment_count
FROM "AwsDataCatalog"."a05"."a05"
GROUP BY subreddit
ORDER BY comment_count DESC
LIMIT 10;
'''

start_time = time.time()
response = athena_client.start_query_execution(
    QueryString=q,
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

logger.info(response)
df_top_subreddits = download_and_load_query_results(athena_client, response)


output_file = "prob1_results.csv"
df_top_subreddits.to_csv(output_file, index=False)
logger.info(f"Results saved to {output_file}")
logger.info(f"Data fetched and committed in {time.time() - start_time}s")

[2024-11-11 03:01:04,936] p537 {2703032465.py:20} INFO - {'QueryExecutionId': '5903af11-2510-4e67-9a7d-3293483c7964', 'ResponseMetadata': {'RequestId': 'a0bdd02d-16a0-4a22-94c8-d1a8b3f5bb3b', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Mon, 11 Nov 2024 03:01:04 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '59', 'connection': 'keep-alive', 'x-amzn-requestid': 'a0bdd02d-16a0-4a22-94c8-d1a8b3f5bb3b'}, 'RetryAttempts': 0}}
[2024-11-11 03:01:04,937] p537 {1241576256.py:4} INFO - download_and_load_query_results, enter
[2024-11-11 03:01:07,569] p537 {1241576256.py:17} INFO - Time to complete query: 2.7114787101745605s
[2024-11-11 03:01:07,585] p537 {1241576256.py:24} INFO - downloading file from S3_BUCKET_NAME=athena-ek976, s3_path=data/results/5903af11-2510-4e67-9a7d-3293483c7964.csv, to local file athena_query_results.csv
[2024-11-11 03:01:07,707] p537 {1241576256.py:31} INFO - results dataframe shape is (10, 2)
[2024-11-11 03:01:07,718] p537 {2703032465.py:26} 

## Problem 2: Get 10 random rows from the comments table
This will help you understand the schema of the table which will be useful for other queries.
Save the results in a file called `prob2_results.csv`. This needs to be commited to the repo.

In [9]:
# Problem 2: Get 10 random rows from the comments table
q = '''
SELECT * 
FROM "AwsDataCatalog"."a05"."a05"
ORDER BY RAND()
LIMIT 10;
'''

start_time = time.time()
response = athena_client.start_query_execution(
    QueryString=q,
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

logger.info(response)
df_random_rows = download_and_load_query_results(athena_client, response)


output_file = "prob2_results.csv"
df_random_rows.to_csv(output_file, index=False)
logger.info(f"Results saved to {output_file}")

logger.info(f"Data fetched and committed in {time.time() - start_time}s")

[2024-10-22 14:46:08,027] p387 {4136649772.py:19} INFO - {'QueryExecutionId': '368fd56b-4aed-46cc-8bdb-850b8164afab', 'ResponseMetadata': {'RequestId': 'c6acaccc-a525-4329-bb73-f1f4d37b4f68', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Tue, 22 Oct 2024 14:46:08 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '59', 'connection': 'keep-alive', 'x-amzn-requestid': 'c6acaccc-a525-4329-bb73-f1f4d37b4f68'}, 'RetryAttempts': 0}}
[2024-10-22 14:46:08,027] p387 {1241576256.py:4} INFO - download_and_load_query_results, enter
[2024-10-22 14:46:21,548] p387 {1241576256.py:17} INFO - Time to complete query: 13.60271167755127s
[2024-10-22 14:46:21,556] p387 {1241576256.py:24} INFO - downloading file from S3_BUCKET_NAME=athena-ek976, s3_path=data/results/368fd56b-4aed-46cc-8bdb-850b8164afab.csv, to local file athena_query_results.csv
[2024-10-22 14:46:21,651] p387 {1241576256.py:31} INFO - results dataframe shape is (10, 17)
[2024-10-22 14:46:21,656] p387 {4136649772.py:25} 

## Problem 3: Get number of comments per day per hour and sort the results in descending order of the count

The response should contain the following 3 columns: `comment_date`, `comment_hour` and `comment_count`.
Save the results in a file called `prob3_results.csv`. This needs to be commited to the repo.

In [10]:
# Problem 3: Get number of comments per day per hour and sort the results in descending order of the count
q = '''
    SELECT 
        CAST(from_unixtime(created_utc) as date) as comment_date,
        HOUR(from_unixtime(created_utc)) as comment_hour,
        COUNT(*) AS comment_count
    FROM
        "AwsDataCatalog"."a05"."a05"
    GROUP BY 
        CAST(from_unixtime(created_utc) as date), 
        HOUR(from_unixtime(created_utc))
    ORDER BY 
        comment_count DESC;
'''


start_time = time.time()
response = athena_client.start_query_execution(
    QueryString=q,
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

logger.info(response)


df_comments_per_hour = download_and_load_query_results(athena_client, response)


output_file = "prob3_results.csv"
df_comments_per_hour.to_csv(output_file, index=False)
logger.info(f"Results saved to {output_file}")

logger.info(f"Data fetched and committed in {time.time() - start_time}s")

[2024-10-22 14:46:21,738] p387 {862638526.py:27} INFO - {'QueryExecutionId': 'ec5fe369-e40b-44a8-ad4e-9f412ae5f566', 'ResponseMetadata': {'RequestId': '19144222-7a42-4100-9809-6b42152085db', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Tue, 22 Oct 2024 14:46:21 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '59', 'connection': 'keep-alive', 'x-amzn-requestid': '19144222-7a42-4100-9809-6b42152085db'}, 'RetryAttempts': 0}}
[2024-10-22 14:46:21,738] p387 {1241576256.py:4} INFO - download_and_load_query_results, enter
[2024-10-22 14:46:23,418] p387 {1241576256.py:17} INFO - Time to complete query: 1.7490768432617188s
[2024-10-22 14:46:23,444] p387 {1241576256.py:24} INFO - downloading file from S3_BUCKET_NAME=athena-ek976, s3_path=data/results/ec5fe369-e40b-44a8-ad4e-9f412ae5f566.csv, to local file athena_query_results.csv
[2024-10-22 14:46:23,542] p387 {1241576256.py:31} INFO - results dataframe shape is (18, 3)
[2024-10-22 14:46:23,547] p387 {862638526.py:35} IN

## Problem 4: Find top 10 subreddits by the highest average score and sort the results in descending order of the average score

The response should contain the following columns: `subreddit` and `avg_score`.
Save the results in a file called `prob4_results.csv`. This needs to be commited to the repo.


In [11]:
# Problem 4: Find top 10 subreddits by the highest average score and sort the results in descending 
# order of the average score
q = '''
SELECT 
    subreddit,
    AVG(score) AS avg_score
FROM 
    "AwsDataCatalog"."a05"."a05"
GROUP BY 
    subreddit
ORDER BY 
    avg_score DESC
LIMIT 10;
'''


start_time = time.time()
response = athena_client.start_query_execution(
    QueryString=q,
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

logger.info(response)


df_avg_scores = download_and_load_query_results(athena_client, response)


output_file = "prob4_results.csv"
df_avg_scores.to_csv(output_file, index=False)
logger.info(f"Results saved to {output_file}")



logger.info(f"Data fetched and committed in {time.time() - start_time}s")


[2024-10-22 14:46:23,629] p387 {1747469377.py:27} INFO - {'QueryExecutionId': 'fbf64ebf-6a73-4cb8-9a33-8dd7a98b27f0', 'ResponseMetadata': {'RequestId': 'f862e7c3-ca8c-4ea0-bb3a-77198017e851', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Tue, 22 Oct 2024 14:46:23 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '59', 'connection': 'keep-alive', 'x-amzn-requestid': 'f862e7c3-ca8c-4ea0-bb3a-77198017e851'}, 'RetryAttempts': 0}}
[2024-10-22 14:46:23,630] p387 {1241576256.py:4} INFO - download_and_load_query_results, enter
[2024-10-22 14:46:25,440] p387 {1241576256.py:17} INFO - Time to complete query: 1.8752353191375732s
[2024-10-22 14:46:25,459] p387 {1241576256.py:24} INFO - downloading file from S3_BUCKET_NAME=athena-ek976, s3_path=data/results/fbf64ebf-6a73-4cb8-9a33-8dd7a98b27f0.csv, to local file athena_query_results.csv
[2024-10-22 14:46:25,542] p387 {1241576256.py:31} INFO - results dataframe shape is (10, 2)
[2024-10-22 14:46:25,550] p387 {1747469377.py:35} 

In [12]:
# Problem 5: Top 5 most Controversial Comments in a r\datascience subreddit
q = '''
SELECT 
    author,
    body,
    score,
    controversiality
FROM 
    "AwsDataCatalog"."a05"."a05"
WHERE 
    subreddit = 'datascience'
ORDER BY 
    controversiality DESC, score DESC
LIMIT 5;
'''


start_time = time.time()
response = athena_client.start_query_execution(
    QueryString=q,
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

logger.info(response)


df_controversial_comments = download_and_load_query_results(athena_client, response)


output_file = "prob5_results.csv"
df_controversial_comments.to_csv(output_file, index=False)
logger.info(f"Results saved to {output_file}")


logger.info(f"Data fetched and committed in {time.time() - start_time}s")

[2024-10-22 14:46:33,117] p387 {1740892584.py:28} INFO - {'QueryExecutionId': '4be8a71c-f0af-40d5-a55a-6fdd32490bfc', 'ResponseMetadata': {'RequestId': '833810bb-65d6-488f-be41-71646292e5bc', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Tue, 22 Oct 2024 14:46:33 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '59', 'connection': 'keep-alive', 'x-amzn-requestid': '833810bb-65d6-488f-be41-71646292e5bc'}, 'RetryAttempts': 0}}
[2024-10-22 14:46:33,119] p387 {1241576256.py:4} INFO - download_and_load_query_results, enter
[2024-10-22 14:46:37,220] p387 {1241576256.py:17} INFO - Time to complete query: 4.169274091720581s
[2024-10-22 14:46:37,229] p387 {1241576256.py:24} INFO - downloading file from S3_BUCKET_NAME=athena-ek976, s3_path=data/results/4be8a71c-f0af-40d5-a55a-6fdd32490bfc.csv, to local file athena_query_results.csv
[2024-10-22 14:46:37,306] p387 {1241576256.py:31} INFO - results dataframe shape is (5, 4)
[2024-10-22 14:46:37,311] p387 {1740892584.py:36} IN